# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Korhonen  ->  H. Korhonen  |  ['H. Korhonen']
S. Kraus  ->  S. Kraus  |  ['S. Kraus']
D. Mortimer  ->  D. Mortimer  |  ['D. Mortimer']
U. Dudzeviciute  ->  U. Dudzeviciute  |  ['U. Dudzeviciute']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 56 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.09676


extracting tarball to tmp_2412.09676...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.09720


extracting tarball to tmp_2412.09720...

...

Found 53 bibliographic references in tmp_2412.09783/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2412.09823


extracting tarball to tmp_2412.09823...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.09864


extracting tarball to tmp_2412.09864...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09737-b31b1b.svg)](https://arxiv.org/abs/2412.09737) | **ALMA/SCUBA-2 COSMOS Survey: Properties of X-ray- and SED-selected AGNs in Bright Submillimeter Galaxies**  |
|| R. Uematsu, et al. -- incl., <mark>U. Dudzeviciute</mark> |
|*Appeared on*| *2024-12-16*|
|*Comments*| *37 pages, 21 figures, accepted for The Astrophysical Journal*|
|**Abstract**|            We investigate the properties of active galactic nuclei (AGNs) in the brightest submillimeter galaxies (SMGs) in the COSMOS field. We utilize the bright sample of ALMA/SCUBA-2 COSMOS Survey (AS2COSMOS), which consists of 260 SMGs with $S_{\mathrm{870}\, \mu \mathrm{m}}=0.7\text{--}19.2\,\mathrm{mJy}$ at $z=0\text{--}6$. We perform optical to millimeter spectral energy distribution (SED) modeling for the whole sample. We identify 24 AGN-host galaxies from the SEDs. Supplemented by 23 X-ray detected AGNs (X-ray AGNs), we construct an overall sample of 40 AGN-host galaxies. The X-ray luminosity upper bounds indicate that the X-ray undetected SED-identified AGNs are likely to be nearly Compton thick or have unusually suppressed X-ray emission. From visual classification, we identify $25^{+6}_{-5}$\% of the SMGs without AGNs as major merger candidates. This fraction is almost consistent with the general galaxy population at $z\sim2$, suggesting that major mergers are not necessarily required for the enhanced star formation in SMGs. We also identify $47^{+16}_{-15}$\% of the AGN hosts as major merger candidates, which is about twice as high as that in the SMGs without AGNs. This suggests that major mergers play a key role in triggering AGN activity in bright SMGs.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09783-b31b1b.svg)](https://arxiv.org/abs/2412.09783) | **oMEGACat V: Helium Enrichment in $\omega$ Centauri as a Function of Metallicity**  |
|| <mark>C. Clontz</mark>, et al. -- incl., <mark>M. Häberle</mark>, <mark>N. Neumayer</mark> |
|*Appeared on*| *2024-12-16*|
|*Comments*| **|
|**Abstract**|            Constraining the helium enhancement in stars is critical for understanding the formation mechanisms of multiple populations in star clusters. However, measuring helium variations for many stars within a cluster remains observationally challenging. We use Hubble Space Telescope photometry combined with MUSE spectroscopic data for over 7,200 red-giant branch stars in \omc\ to measure helium differences between distinct groups of stars as a function of metallicity separating the impact of helium enhancements from other abundance variations on the pseudo-color (chromosome) diagrams. Our results show that stars at all metallicities have subpopulations with significant helium enhancement ($\Delta Y_{min} \gtrsim$ 0.11). We find a rapid increase in helium enhancement from low metallicities ($\rm{[Fe/H] \simeq -2.05}$ to $\rm{[Fe/H] \simeq -1.92})$, with this enhancement leveling out at \deltay\ $= 0.154$ at higher metallicities. The fraction of helium-enhanced stars steadily increases with metallicity ranging from 10\% at $\rm{[Fe/H] \simeq -2.04}$ to over $90\%$ at $\rm{[Fe/H] \simeq -1.04}$. This study is the first to examine helium enhancement across the full range of metallicities in \omc{}, providing new insight into its formation history and additional constraints on enrichment mechanisms.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09720-b31b1b.svg)](https://arxiv.org/abs/2412.09720) | **Newborn Be star systems observed shortly after mass transfer**  |
|| T. Rivinius, et al. -- incl., <mark>S. Kraus</mark>, <mark>D. Mortimer</mark> |
|*Appeared on*| *2024-12-16*|
|*Comments*| *Accepted for publication by A&A*|
|**Abstract**|            Many classical Be stars acquire their very rapid rotation by mass and angular-momentum transfer in massive binaries. Short-lived intermediate-phase objects have only been discovered recently. Data archives and the literature have been searched for additional candidates exhibiting this patterns. Thirteen candidates were identified at various confidence levels. Adding to the two known systems identified as classical Be star+pre-subdwarf binaries (LB-1 and HR6819), two more (V742Cas, HD44637) could be confirmed with interferometry, with V742Cas setting a new record for the smallest visually observed angular semi-major axis, at a=0.663mas. Two further ones (V447Sct, V1362Cyg) are not resolved interferometrically, but other evidence puts them at the same confidence level as LB-1. V2174Cyg is a candidate with very high confidence, but was not observed interferometrically. The remaining ones are either candidates with varying levels of confidence. Of a mostly magnitude complete sample of 328 Be stars, 0.5-1% are found to have recently completed the mass overflow that led to their formation. Another 5% are systems with compact subdwarf companions, i.e., further evolved after a previous overflow, and possibly two more percent harbor white dwarfs. All these systems are of early B-subtypes, however, and if the original sample is restricted to early subtypes (136 objects), these percentages increase by a factor of about 2.5, while dropping to zero for the mid and late subtypes (together 204 objects). This strongly suggests that early- vs. mid- and late-type Be stars have differently weighted channels to acquire their rapid rotation, namely binary interaction vs. evolutionary spin-up.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09823-b31b1b.svg)](https://arxiv.org/abs/2412.09823) | **3 mm Spectroscopic Observations of Massive Star-Forming Regions with IRAM 30-m**  |
|| X. Xu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-12-16*|
|*Comments*| *22 pages, 3 figures, 14 tables, accepted for publication in PASJ*|
|**Abstract**|            Broadband spectroscopic observations with high sensitivity provide an unbiased way to detect emissions of molecules in space. We present deep observations from ~ 105.8 GHz to 113.6 GHz toward 50 Galactic massive star-forming regions using IRAM 30-m millimeter telescope, with noise levels ranging from 6 to 29 at frequency channel spacing of 195 kHz, which corresponds to ~ 0.54 km/s at 110 GHz. Totally, 27 molecular species have been identified, of which 16 are complex organic molecules. The related parameters, such as peak temperature, integrated intensity, and line width of the identified molecular lines were obtained. The line widths of the chemically related molecules show strong positive correlations, suggesting they likely originate from similar gases within star-forming regions. This work highlights the fundamental properties of the detected molecular lines and offers a valuable dataset for further studies on the astrochemical evolution of molecules in massive star-forming cores.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09864-b31b1b.svg)](https://arxiv.org/abs/2412.09864) | **The Three Hundred Project: The relationship between the shock and splashback radii of simulated galaxy clusters**  |
|| M. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-16*|
|*Comments*| *11 pages, 6 figures; accepted for publication in the Publications of the Astronomical Society of Australia (PASA)*|
|**Abstract**|            Observations of the intracluster medium (ICM) in the outskirts of galaxy clusters reveal shocks associated with gas accretion from the cosmic web. Previous work based on non-radiative cosmological hydrodynamical simulations have defined the shock radius, $r_\text{shock}$, using the ICM entropy, $K \propto T/{n_\mathrm{e}}^{2/3}$, where $T$ and $n_\text{e}$ are the ICM temperature and electron density respectively; the $r_\text{shock}$ is identified with either the radius at which $K$ is a maximum or at which its logarithmic slope is a minimum. We investigate the relationship between $r_\text{shock}$, which is driven by gravitational hydrodynamics and shocks, and the splashback radius, $r_\text{splash}$, which is driven by the gravitational dynamics of cluster stars and dark matter and is measured from their mass profile. Using 324 clusters from {\small The Three Hundred} project of cosmological galaxy formation simulations, we quantify statistically how $r_\text{shock}$ relates to $r_\text{splash}$. Depending on our definition, we find that the median $r_\text{shock} \simeq 1.38 r_\text{splash} (2.58 R_{200})$ when $K$ reaches its maximum and $r_\text{shock} \simeq 1.91 r_\text{splash} (3.54 R_{200})$ when its logarithmic slope is a minimum; the best-fit linear relation increases as $r_\text{shock} \propto 0.65 r_\text{splash}$. We find that $r_\text{shock}/R_{200}$ and $r_\text{splash}/R_{200}$ anti-correlate with virial mass, $M_{200}$, and recent mass accretion history, and $r_\text{shock}/r_\text{splash}$ tends to be larger for clusters with higher recent accretion rates. We discuss prospects for measuring $r_\text{shock}$ observationally and how the relationship between $r_\text{shock}$ and $r_\text{splash}$ can be used to improve constraints from radio, X-ray, and thermal Sunyaev-Zeldovich surveys that target the interface between the cosmic web and clusters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.09676-b31b1b.svg)](https://arxiv.org/abs/2412.09676) | **OGLE-2015-BLG-1609Lb: Sub-jovian planet orbiting a low-mass stellar or brown dwarf host**  |
|| M. Mróz, et al. -- incl., <mark>H. Korhonen</mark> |
|*Appeared on*| *2024-12-16*|
|*Comments*| *12 pages, 7 figures*|
|**Abstract**|            We present a comprehensive analysis of a planetary microlensing event OGLE-2015-BLG-1609. The planetary anomaly was detected by two survey telescopes, OGLE and MOA. Each of these surveys collected enough data over the planetary anomaly to allow for an unambiguous planet detection. Such survey detections of planetary anomalies are needed to build a robust sample of planets that could improve studies on the microlensing planetary occurrence rate by reducing biases and statistical uncertainties. In this work, we examined different methods for modeling microlensing events using individual datasets, particularly we incorporated a Galactic model prior to better constrain poorly defined microlensing parallax. Ultimately, we fitted a comprehensive model to all available data, identifying three potential typologies, with two showing comparably high Bayesian evidence. Our analysis indicates that the host of the planet is a brown dwarf with a probability of 34%, or a low-mass stellar object (M-dwarf) with the probability of 66%.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.09737/./grid_miri.png', 'tmp_2412.09737/./hist_separation.png', 'tmp_2412.09737/./ximage.png']
copying  tmp_2412.09737/./grid_miri.png to _build/html/
copying  tmp_2412.09737/./hist_separation.png to _build/html/
copying  tmp_2412.09737/./ximage.png to _build/html/
exported in  _build/html/2412.09737.md
    + _build/html/tmp_2412.09737/./grid_miri.png
    + _build/html/tmp_2412.09737/./hist_separation.png
    + _build/html/tmp_2412.09737/./ximage.png
found figures ['tmp_2412.09783/./figures/helium_ruler_creation_3_panels_v3.png', 'tmp_2412.09783/./figures/helium_ruler_results.png', 'tmp_2412.09783/./figures/stream_wise_results_2_panel_v3.png', 'tmp_2412.09783/./figures/chromosome_map_colored_by_feh_v3.png']
copying  tmp_2412.09783/./figures/helium_ruler_creation_3_panels_v3.png to _build/html/
copying  tmp_2412.09783/./figures/helium_ruler_results.png to _build/html/
copying  tmp_2412.09783/./figures/stream_wise_results_2_panel_v3.png to _build/html/
copying  t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# ALMA/SCUBA-2 COSMOS Survey: Properties of X-ray- and SED-selected AGNs in Bright Submillimeter Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.09737-b31b1b.svg)](https://arxiv.org/abs/2412.09737)<mark>Appeared on: 2024-12-16</mark> -  _37 pages, 21 figures, accepted for The Astrophysical Journal_

</div>
<div id="authors">

R. Uematsu, et al.

</div>
<div id="abstract">

**Abstract:** We investigate the properties of active galactic nuclei (AGNs) in the brightest submillimeter galaxies (SMGs) in the COSMOS field. We utilize the bright sample of ALMA/SCUBA-2 COSMOS Survey (AS2COSMOS), which consists of 260 SMGs with $S_{\mathrm{870}  \mu \mathrm{m}}=0.7\text{--}19.2 \mathrm{mJy}$ at $z=0\text{--}6$ . We perform optical to millimeter spectral energy distribution (SED) modeling for the whole sample. We identify 24 AGN-host galaxies from the SEDs. Supplemented by 23 X-ray detected AGNs (X-ray AGNs), we construct an overall sample of 40 AGN-host galaxies. The X-ray luminosity upper bounds indicate that the X-ray undetected SED-identified AGNs are likely to be nearly Compton thick or have unusually suppressed X-ray emission. From visual classification, we identify $25^{+6}_{-5}$ \% of the SMGs without AGNs as major merger candidates. This fraction is almost consistent with the general galaxy population at $z\sim2$ , suggesting that major mergers are not necessarily required for the enhanced star formation in SMGs. We also identify $47^{+16}_{-15}$ \% of the AGN hosts as major merger candidates, which is about twice as high as that in the SMGs without AGNs. This suggests that major mergers play a key role in triggering AGN activity in bright SMGs.

</div>

<div id="div_fig1">

<img src="tmp_2412.09737/./grid_miri.png" alt="Fig15" width="100%"/>

**Figure 15. -** 
  8 arcsec $\times$ 8 arcsec JWST images of the 40 AS2COSMOS sources in the coverage of both NIRCam and MIRI imaging. The blue, green, and red colors correspond to the F115W+F150W, F277W+F444W, and F770W filters, respectively. We label the merger candidates, which have tidal features (T), disturbed morphology (D) or possible companions (C) (see Section \ref{subsection:morphology}). The major merger candidates are indicated by "M". The X-ray AGNs and the SED AGNs are indicated by "X" and "SED" (see Section \ref{subsubsection:agn_identification} and Section \ref{subsubsection:x-ray_LFIR}). Note that F150W is not used in the composite image of AS2COS0107.1, because this object is outside the coverage of F150W. For the same reason, F444W is not used in the composite image of AS2COS155.1. (*figure:JWST_miri*)

</div>
<div id="div_fig2">

<img src="tmp_2412.09737/./hist_separation.png" alt="Fig2" width="100%"/>

**Figure 2. -** _(a)_
  Positional offsets between optical sources and X-ray sources used for the astrometry correction. The solid, dotted, and dashed circles encompass 68\%, 90\%, and 95\% of the sources before (red) and after (blue) the correction, respectively. _(b)_ Histogram of the separation between the X-ray source positions and the optical to near-infrared source positions in the COSMOS2020 catalog. The red area shows the separation between \citet{2016ApJ...819...62C} and the COSMOS2020 catalog, while the blue area denotes the separation between our X-ray source catalog (Section \ref{subsubsection:x-ray_extraction}) and the COSMOS2020 catalog. (*figure:separation*)

</div>
<div id="div_fig3">

<img src="tmp_2412.09737/./ximage.png" alt="Fig3" width="100%"/>

**Figure 3. -** Positions of the X-ray detected sources near AS2COS0353.1 and AS2COS0353.2 plotted over the Ultravista $K_{\mathrm{s}}$-band image. The red points show the X-ray source positions listed in the catalog by \citet{2016ApJ...819...62C}, whereas the blue points show the positions derived in our analysis (Section \ref{subsubsection:x-ray_extraction}). The green points show the positions of AS2COS0353.1 and AS2COS0353.2. The bottom panels show the zoomed-in images of the X-ray sources. (*figure:ximage*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.09737"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# oMEGACat V: Helium Enrichment in $\omega$ Centauri as a Function of Metallicity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.09783-b31b1b.svg)](https://arxiv.org/abs/2412.09783)<mark>Appeared on: 2024-12-16</mark> - 

</div>
<div id="authors">

<mark>C. Clontz</mark>, et al. -- incl., <mark>M. Häberle</mark>, <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** Constraining the helium enhancement in stars is critical for understanding the formation mechanisms of multiple populations in star clusters. However, measuring helium variations for many stars within a cluster remains observationally challenging. We use Hubble Space Telescope photometry combined with MUSE spectroscopic data for over 7,200 red-giant branch stars in $\omc$ to measure helium differences between distinct groups of stars as a function of metallicity separating the impact of helium enhancements from other abundance variations on the pseudo-color (chromosome) diagrams. Our results show that stars at all metallicities have subpopulations with significant helium enhancement ( $\Delta Y_{min} \gtrsim$ 0.11). We find a rapid increase in helium enhancement from low metallicities ( $\rm{[Fe/H] \simeq -2.05}$ to $\rm{[Fe/H] \simeq -1.92})$ , with this enhancement leveling out at $\deltay$ $= 0.154$ at higher metallicities. The fraction of helium-enhanced stars steadily increases with metallicity ranging from 10 \% at $\rm{[Fe/H] \simeq -2.04}$ to over $90\%$ at $\rm{[Fe/H] \simeq -1.04}$ . This study is the first to examine helium enhancement across the full range of metallicities in $\omc$ , providing new insight into its formation history and additional constraints on enrichment mechanisms.

</div>

<div id="div_fig1">

<img src="tmp_2412.09783/./figures/helium_ruler_creation_3_panels_v3.png" alt="Fig6.1" width="50%"/><img src="tmp_2412.09783/./figures/helium_ruler_results.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** **Helium Ruler vs. [Fe/H]:*** Left Panel --* These three figures show the derivation of the helium ruler. First on the far left, the color magnitude-diagram of two isochrones [Fe/H]$= -1.70$, but with primordial (red) and enhanced (blue) helium abundance are shown. The yellow and green lines show the fiducials used to verticalize the color-magnitude diagram and create the \deltaone color shown in panel (b). The black arrows in this panel show the distance between the verticalized isochrones in each magnitude bin. The histogram in panel (c) shows the number of stars at each magnitude that are used to weight the separation of the helium isochrones and obtain the helium ruler x-distance, He$_{ruler,x}$. The gray region at the top of the panel (b) shows magnitudes excluded from our analysis to ensure consistent length helium ruler vectors for all stars; the rulers at these magnitudes are clearly shorter. * Right Panel --* The derived He$_{ruler,x}$ in the \deltaone color as a function of metallicity. This corresponds to the expected offset in the \deltaone\/x-axis of the chromosome diagram for two populations with $\Delta Y \sim 0.15$. This difference is calculated at each metallicity across the full range of RGB magnitudes; the dark blue points show the He$_{ruler,x}$ determined from the weighted average of all the vectors in the left-middle panel. The blue band shows the 16th and 84th percentile of the vector lengths. (*fig:helium_ruler_creation_and_results*)

</div>
<div id="div_fig2">

<img src="tmp_2412.09783/./figures/stream_wise_results_2_panel_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Helium Enhancement vs. Metallicity:**(_Top panel_) The fraction of stars in each of the three streams defined in Fig. \ref{fig:chrom_map} as a function of metallicity (gold, blue, and red lines). Gray lines show the relative numbers of stars in the two different age-metallicity relationship tracks found by [Clontz, Seth and Dotter (2024)]().
(_Bottom panel_) \deltay for the lower-to-middle stream (blue/red points) and the lower-to-upper stream (blue/gold points) as a function of metallicity. The highest metallicity points are shown at low opacity because their results are very uncertain and are based on a small number of stars ($<$30).
 (*fig:streamwise_results*)

</div>
<div id="div_fig3">

<img src="tmp_2412.09783/./figures/chromosome_map_colored_by_feh_v3.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Chromosome/Pseudo-color diagram**: Our sample of 7,277 RGB stars in \omc is shown with each star colored by its metallicity. We separate the RGB stars separate into three distinct streams in this diagram using the diagonal black lines and color-coded labels. The edges of the [Fe/H] bins are indicated by white lines on the color bar (see also Table \ref{table:results}) Spreads in the \deltaone and \deltatwo within metallicity bins are primarily due to light element abundance variations. (*fig:chrom_map*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.09783"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

283  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
